In [6]:
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
import numpy as np
from source.FeatureSelectionClasses import rfe, lasso, pi
from source.ClassifierClasses import LogRegression, NeuralNet, RandomForest
from SimPy.Statistics import SummaryStat

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix
## read data 
CIP_data = pd.read_csv("CIP_data_encode_prev.csv")
CIP_data.head()
print(CIP_data.columns)

ModuleNotFoundError: No module named 'source.FeatureSelectionClasses'; 'source' is not a package

In [3]:

def get_mean_min_max(string):
    split_list = re.split('[(,)]', string)
    mean = float(split_list[0])
    min_value = float(split_list[1])
    max_value = float(split_list[2])
    return mean, min_value, max_value


def subtract_two_lists(list1, list2):
    difference = []
    zip_object = zip(list1, list2)
    for list1_i, list2_i in zip_object:
        difference.append(list1_i - list2_i)
    return difference


def get_pfm_with_same_p(list_of_list):
    """ get list of optimisms of fpr/tpr under the same classification threshold (p) """
    list_to_array = np.array(list_of_list)
    transposed_array = list_to_array.T
    transposed_list = transposed_array.tolist()
    return transposed_list


def get_performance_helper(app_pfm, opti_distr):
    """ calculate mean and confidence interval for optimism-corrected performance
    :param app_pfm: (float) a constant number, apparent performance
    :param opti_distr: (string) mean (min, max), a distribution
    :return (string) pfm_mean [pfm_min, pfm_max]
    """
    mean, min_value, max_value = get_mean_min_max(opti_distr)
    performance_mean = round((app_pfm - mean), 2)
    performance_min = round((app_pfm - max_value), 2)
    performance_max = round((app_pfm - min_value), 2)
    performance_output = '{} ({},{})'.format(performance_mean, performance_min, performance_max)
    return performance_output


def get_utility_helper(component1_distr, component2_distr, tradeoff):
    """
    calculate utility and 95% confidence interval based on tradeoff threshold
    :param component1_distr: (string) mean (min, max) of sensitivity * flq_prev
    :param component2_distr: (string) mean (min, max) of (1 - specificity) * (1 - flq_prev)
    :param tradeoff: policy makers' trade-off threshold
    :return: (string) mean (min, max) of utility
    """
    mean1, min1, max1 = get_mean_min_max(component1_distr)
    mean2, min2, max2 = get_mean_min_max(component2_distr)
    performance_mean = round(tradeoff * mean1 + mean2, 2)
    performance_min = round(tradeoff * min1 + max2, 2)
    performance_max = round(tradeoff * max1 + min2, 2)
    performance_output = '{} ({},{})'.format(performance_mean, performance_mean, performance_mean)
    return performance_output


class BootstrapModel:
    def __init__(self, df, y_name, classifier, feature_selection=None,
                 nn_activation='tanh', nn_solver='lbfgs', nn_alpha=2,  # NN combination of best performance
                 rf_ntree=100, rf_min_leaf=5,                          # RF combination of best performance
                 num_selected_features=10, lasso_penalty=0.1, threshold=0.5, tradeoff=None, tradeoff_list=None, flq_prev=None):
        """
        calculate the optimism-corrected performance based on specified feature selection and classifier
        :param df: DataFrame used for model construction
        :param y_name: outcome of interests
        :param feature_selection: "PI": permutation importance,
                                  "RFE": recursive feature selection,
                                  "LASSO": L1 regularization
        :param classifier: "logistic": logistic regression
                           "neural": neural network
                           "rf": random forest
        :param num_selected_features: specify number of features wanted, cannot coexist with penalty
        :param lasso_penalty: hyper-parameter for LASSO feature selection method, cannot coexist with num_selected_features
        :param threshold: classification threshold
        :param tradeoff: policy makers' utility towards two different scenarios proposed in the paper
        :param flq_prev: prevalence of resistance to FLQs in the whole dataset
        """
        self.df = df                                        # the whole dataset
        self.y_name = y_name                                # outcome of interest
        self.threshold = threshold                          # classification threshold
        self.tradeoff = tradeoff
        self.tradeoff_list = tradeoff_list
        self.flq_prev = flq_prev
        self.classifier = classifier
        self.feature_selection = feature_selection
        self.num_selected_features = num_selected_features
        self.penalty = lasso_penalty                              # if LASSO is specified, use this than num_selected_features
        self.original_apparent_pfm = None                   # apparent performance using the whole dataset
        self.corrected_pfm = None                           # optimism-corrected performance
        self.predictor_counts = dict()                      # recording frequency of features identified as significant
        # For NN
        self.nn_activation = nn_activation
        self.nn_solver = nn_solver
        self.nn_alpha = nn_alpha
        # For RF
        self.rf_ntree = rf_ntree
        self.rf_min_leaf = rf_min_leaf

    def select_significant_features(self, df):
        """ select features based on the given dataset, the feature selection method and the classifier """
        if self.feature_selection is None:
            significant_features = df.columns.tolist()
            significant_features.remove(self.y_name)
        else:
            # logistic regression, feature selection method include PI, RFE, and LASSO
            if self.classifier == 'logistic':
                if self.feature_selection == 'LASSO':
                    significant_features = lasso(df=df, y_name=self.y_name, penalty=self.penalty)
                elif self.feature_selection == 'PI':
                    significant_features = pi(df=df, y_name=self.y_name, classifier='logistic',
                                              num_features=self.num_selected_features)
                elif self.feature_selection == 'RFE':
                    significant_features = rfe(df=df, y_name=self.y_name, classifier='logistic',
                                               num_features=self.num_selected_features)
                else:
                    raise ValueError('invalid feature selection method for logistic regression model')
            # neural network model, only PI is applicable
            elif self.classifier == 'neural':
                if self.feature_selection == 'PI':
                    significant_features = pi(df=df, y_name=self.y_name, classifier='neural',
                                              num_features=self.num_selected_features,
                                              nn_alpha=self.nn_alpha, nn_solver=self.nn_solver,
                                              nn_activation=self.nn_activation)
                else:
                    raise ValueError('invalid feature selection method for neural network model')
            # random forest, applicable feature selection methods include: PI and RFE
            elif self.classifier == 'rf':
                if self.feature_selection == 'PI':
                    significant_features = pi(df=df, y_name=self.y_name, classifier='rf',
                                              rf_ntree=self.rf_ntree, rf_min_leaf=self.rf_min_leaf,
                                              num_features=self.num_selected_features)
                elif self.feature_selection == 'RFE':
                    significant_features = rfe(df=df, y_name=self.y_name, classifier='rf',
                                               rf_ntree=self.rf_ntree, rf_min_leaf=self.rf_min_leaf,
                                               num_features=self.num_selected_features)
                else:
                    raise ValueError('invalid feature selection method for random forest model')
            else:
                raise ValueError('invalid classifier')
        return significant_features

    def _get_origin_apparent_pfm(self, display_roc_curve=False):
        """ use the whole dataset to train and evaluate the obtained model """
        original_df = self.df
        # feature selection
        significant_features = self.select_significant_features(df=original_df)
        print('features selected for whole dataset:', significant_features)
        # train and test using the whole dataset
        # specify classification algorithm
        if self.classifier == 'logistic':
            predict_model = LogRegression(features=significant_features, y_name=self.y_name)
            predict_model.run(df_train=original_df, df_test=original_df, display_roc_curve=display_roc_curve,
                              threshold=self.threshold, flq_prev=self.flq_prev)
        elif self.classifier == 'neural':
            predict_model = NeuralNet(features=significant_features, y_name=self.y_name)
            # just for NN
            predict_model.run(df_train=original_df, df_test=original_df, display_roc_curve=display_roc_curve,
                              activation=self.nn_activation, solver=self.nn_solver, alpha=self.nn_alpha,
                              threshold=self.threshold, flq_prev=self.flq_prev)
        else:
            predict_model = RandomForest(features=significant_features, y_name=self.y_name)
            predict_model.run(df_train=original_df, df_test=original_df, display_roc_curve=display_roc_curve,
                              min_samples_leaf=self.rf_min_leaf, n_trees=self.rf_ntree,
                              threshold=self.threshold, flq_prev=self.flq_prev)
        # run the model
        # predict_model.run(df_train=original_df, df_test=original_df, display_roc_curve=display_roc_curve,
        #                   threshold=self.threshold, flq_prev=self.flq_prev)
        # collect apparent performance
        self.original_apparent_pfm = predict_model.performanceTest

    def _get_bootstrap_pfm(self, rng, display_roc_curve=False):
        """ use the same bootstrap sample to train and evaluate the obtained model """
        # bootstrap sample
        resampled_df = self.df.sample(frac=1, replace=True, random_state=rng)
        # feature selection
        significant_features = self.select_significant_features(df=resampled_df)
        # train and test use the same resampled dataset
        # specify classification algorithm
        if self.classifier == 'logistic':
            bootstrap_model = LogRegression(features=significant_features, y_name=self.y_name)
            bootstrap_model.run(df_train=resampled_df, df_test=resampled_df, display_roc_curve=display_roc_curve,
                                threshold=self.threshold, flq_prev=self.flq_prev)
        elif self.classifier == 'neural':
            bootstrap_model = NeuralNet(features=significant_features, y_name=self.y_name)
            bootstrap_model.run(df_train=resampled_df, df_test=resampled_df, display_roc_curve=display_roc_curve,
                                activation=self.nn_activation, solver=self.nn_solver, alpha=self.nn_alpha,
                                threshold=self.threshold, flq_prev=self.flq_prev)
        else:
            bootstrap_model = RandomForest(features=significant_features, y_name=self.y_name)
            bootstrap_model.run(df_train=resampled_df, df_test=resampled_df, display_roc_curve=display_roc_curve,
                                min_samples_leaf=self.rf_min_leaf, n_trees=self.rf_ntree,
                                threshold=self.threshold, flq_prev=self.flq_prev)
        # run the model
        # bootstrap_model.run(df_train=resampled_df, df_test=resampled_df, display_roc_curve=display_roc_curve,
        #                     threshold=self.threshold, flq_prev=self.flq_prev)
        # collect apparent performance
        bootstrap_apparent_performance = bootstrap_model.performanceTest
        return bootstrap_apparent_performance, significant_features

    def _get_test_pfm(self, rng, display_roc_curve=False):
        """ use bootstrap sample to train, and use the whole dataset to evaluate the obtained model """
        # bootstrap sample (use same random_state, get the same sample set)
        resampled_df = self.df.sample(frac=1, replace=True, random_state=rng)
        # feature selection
        significant_features = self.select_significant_features(df=resampled_df)
        # train use resampled dataset, test use whole dataset
        # specify classification algorithm
        if self.classifier == 'logistic':
            test_model = LogRegression(features=significant_features, y_name=self.y_name)
            test_model.run(df_train=resampled_df, df_test=self.df, display_roc_curve=display_roc_curve,
                           threshold=self.threshold, flq_prev=self.flq_prev)
        elif self.classifier == 'neural':
            test_model = NeuralNet(features=significant_features, y_name=self.y_name)
            test_model.run(df_train=resampled_df, df_test=self.df, display_roc_curve=display_roc_curve,
                           activation=self.nn_activation, solver=self.nn_solver, alpha=self.nn_alpha,
                           threshold=self.threshold, flq_prev=self.flq_prev)
        else:
            test_model = RandomForest(features=significant_features, y_name=self.y_name)
            test_model.run(df_train=resampled_df, df_test=self.df, display_roc_curve=display_roc_curve,
                           min_samples_leaf=self.rf_min_leaf, n_trees=self.rf_ntree,
                           threshold=self.threshold, flq_prev=self.flq_prev)
        # run the model
        # test_model.run(df_train=resampled_df, df_test=self.df, display_roc_curve=display_roc_curve,
        #                threshold=self.threshold, flq_prev=self.flq_prev)
        # collect apparent performance
        test_performance = test_model.performanceTest
        return test_performance

    def _calculate_optimism(self, rng, roc_curve):
        """ calculate optimism """
        bootstrap_pfm, significant_features = self._get_bootstrap_pfm(rng=rng)
        optimism = BootstrapOptimism(bootstrap_performance=bootstrap_pfm,
                                     test_performance=self._get_test_pfm(rng=rng),
                                     roc_curve=roc_curve,
                                     tradeoff=self.tradeoff,
                                     tradeoff_list=self.tradeoff_list,
                                     flq_prev=self.flq_prev)
        return optimism, significant_features

    def get_optimism_corrected_pfm(self, num_bootstrap, predictor_names, plot_optimism_graph=False, roc_curve=False):
        """ calculate optimism-corrected performance (AUC-ROC), plot optimism graph
        :param num_bootstrap: number of bootstrap iterations
        :param predictor_names: list of names of predictors
        :param plot_optimism_graph: whether we want to plot optimism graph
        :param roc_curve: whether we want to calculate stats used for plotting ROC curve
        """
        optimism_list = []
        for feature in predictor_names:
            self.predictor_counts[feature] = 0
        self._get_origin_apparent_pfm()     # update self.original_apparent_pfm
        # append optimism-corrected performance into corresponding list
        for i in range(0, num_bootstrap):
            optimism, significant_features = self._calculate_optimism(rng=i, roc_curve=roc_curve)
            optimism_list.append(optimism)
            for feature in significant_features:
                self.predictor_counts[feature] += 1
        self.corrected_pfm = BootstrapOptimismCorrectedPfm(optimism_list=optimism_list,
                                                           app_pfm=self.original_apparent_pfm,
                                                           # predictor_counts=self.predictor_counts,
                                                           tradeoff=self.tradeoff,
                                                           flq_prev=self.flq_prev,
                                                           tradeoff_list=self.tradeoff_list,
                                                           roc_curve=roc_curve)
        # calculate optimism-corrected performance
        self.corrected_pfm._calculate_optimism_corrected_pfm()
        # plot graph (add area under the curve texts)
        if roc_curve:
            fig = plt.gcf()
            fig.set_size_inches(6, 6)
            plt.legend(loc="lower right")
            plt.text(0.9, 0.1,
                     "Area:{}".format(self.corrected_pfm.auc, 3),
                     ha="right", va="bottom")
            plt.show()
        if plot_optimism_graph:
            self.corrected_pfm.plot_auc_optimism()


class BootstrapOptimism:
    """ optimism values for different performance metrics for one bootstrap sample"""
    def __init__(self, bootstrap_performance, test_performance, roc_curve, flq_prev=None,
                 tradeoff=None, tradeoff_list=None):
        """
        calculate performance optimisms for different metrics
        :param bootstrap_performance: bootstrap apparent performance
        :param test_performance: bootstrap test performance
        :param roc_curve: (boolean) whether want calculate optimism for ROC curve
        :param tradeoff: (int) trade-off threshold (should not coexist with tradeoff_list)
        :param tradeoff_list: (list) a list of trade-off thresholds of interests (should not coexist with tradeoff)
        """
        self.auc = bootstrap_performance.roc_auc - test_performance.roc_auc
        self.sen = bootstrap_performance.sensitivity - test_performance.sensitivity
        self.spe = bootstrap_performance.specificity - test_performance.specificity
        self.F1 = bootstrap_performance.F1 - test_performance.F1
        self.mcc = bootstrap_performance.mcc - test_performance.mcc
        if flq_prev is not None:
            self.effective = bootstrap_performance.receive_effective_regimen - test_performance.receive_effective_regimen
            self.dlm = bootstrap_performance.receive_unnecessary_DML - test_performance.receive_unnecessary_DML
            self.uComponent1 = bootstrap_performance.uComponent1 - test_performance.uComponent1
            self.uComponent2 = bootstrap_performance.uComponent2 - test_performance.uComponent2
        if tradeoff is not None:
            bootstrap_utility = tradeoff * bootstrap_performance.uComponent1 + bootstrap_performance.uComponent2
            test_utility = tradeoff * test_performance.uComponent1 + test_performance.uComponent2
            self.utility = bootstrap_utility - test_utility  # optimism for DeltaUtility
        # use bootstrap/test sensitivity, specificity, and tradeoff threshold to calculate bootstrap and test utility
        if tradeoff_list is not None:
            self.utility_list = []
            for i in tradeoff_list:
                bootstrap_utility = i * bootstrap_performance.uComponent1 + bootstrap_performance.uComponent2
                test_utility = i * test_performance.uComponent1 + test_performance.uComponent2
                self.utility_list.append(bootstrap_utility - test_utility)  # list of optimisms for DeltaUtility
        # calculate optimism for fpr and tpr (a list with various classification threshold)
        if roc_curve:
            self.fpr_list = subtract_two_lists(list1=bootstrap_performance.fpr, list2=test_performance.fpr)
            self.tpr_list = subtract_two_lists(list1=bootstrap_performance.tpr, list2=test_performance.tpr)
            # bootstrap_performance.plot_roc_curve()


class BootstrapOptimismCorrectedPfm:
    """ optimism values for different performance metrics for multiple bootstrap sample
    and the optimism-corrected performance """
    def __init__(self, optimism_list, app_pfm,
                 # predictor_counts,
                 tradeoff=None, tradeoff_list=None, flq_prev=None, roc_curve=True):
        """
        :param optimism_list: a list of BootstrapOptimism
        :param app_pfm: original apparent performances
        # :param predictor_counts: predictor frequency counts
        :param tradeoff: (int) trade-off threshold, for utility calculation
        :param tradeoff_list: (list) of trad-off thresholds, for utility calculation
        :param flq_prev: (float) prevalence of resistance to FLQs
        :param roc_curve: (boolean) whether have parameters for ROC curve
        """
        # input attributes
        self.tradeoff = tradeoff
        self.flq_prev = flq_prev
        self.tradeoff_list = tradeoff_list
        self.optimism_list = optimism_list
        # self.predictor_counts = predictor_counts
        self.roc_curve = roc_curve
        self.original_apparent_performance = app_pfm
        # get summary statistics for optimism values of different performance metrics
        self.stat_opti_auc = SummaryStat(name='optimism for AUC-ROC',
                                         data=[optimism.auc for optimism in self.optimism_list])
        self.stat_opti_sen = SummaryStat(name='optimism for sensitivity',
                                         data=[optimism.sen for optimism in self.optimism_list])
        self.stat_opti_spe = SummaryStat(name='optimism for specificity',
                                         data=[optimism.spe for optimism in self.optimism_list])
        self.stat_opti_F1 = SummaryStat(name='optimism for F1',
                                        data=[optimism.F1 for optimism in self.optimism_list])
        self.stat_opti_mcc = SummaryStat(name='optimism for MCC',
                                         data=[optimism.mcc for optimism in self.optimism_list])
        if self.flq_prev is not None:
            self.stat_opti_dlm = SummaryStat(name='optimism for people receive DML',
                                             data=[optimism.dlm for optimism in self.optimism_list])
            self.stat_opti_effective = SummaryStat(name='optimism for % of people effectively treated',
                                                   data=[optimism.effective for optimism in self.optimism_list])
            self.stat_opti_uComponent1 = SummaryStat(name='optimism for component 1 for utility calculation',
                                                     data=[optimism.uComponent1 for optimism in self.optimism_list])
            self.stat_opti_uComponent2 = SummaryStat(name='optimism for component 2 for utility calculation',
                                                     data=[optimism.uComponent2 for optimism in self.optimism_list])
        if self.tradeoff is not None:
            self.stat_opti_utility = SummaryStat(name='optimism for utility',
                                                 data=[optimism.utility for optimism in self.optimism_list])
            self.utility = None
        if self.tradeoff_list is not None:
            self.uDic = {}
            for i in self.tradeoff_list:
                i = int(i)
                self.uDic['trad-off threshold {}'.format(i)] = \
                    SummaryStat(name='optimism for utilities for trade-off threshold {}'.format(i),
                                data=[optimism.utility_list[i] for optimism in self.optimism_list])
            self.utility_list_by_threshold = []        # [utility_1 (95% CI), ..., utility_m (95% CI)]
        if self.roc_curve:
            # list of list of optimisms for each bootstrap iteration
            list_opti_fpr_list = [optimism.fpr_list for optimism in self.optimism_list]
            list_opti_tpr_list = [optimism.tpr_list for optimism in self.optimism_list]
            # convert to list of list of optimisms under same classification threshold
            list_fpr_same_p_list = get_pfm_with_same_p(list_of_list=list_opti_fpr_list)
            list_tpr_same_p_list = get_pfm_with_same_p(list_of_list=list_opti_tpr_list)
            # print('list_fpr_same_p_list')
            # print(list_fpr_same_p_list)
            # print(len(list_fpr_same_p_list))
            # get list of summary stats for fpr and tpr, with same classification threshold value
            self.list_stat_opti_fpr = [SummaryStat(name='fpr', data=fpr_list) for fpr_list in list_fpr_same_p_list]
            self.list_stat_opti_tpr = [SummaryStat(name='tpr', data=tpr_list) for tpr_list in list_tpr_same_p_list]

        # calculate optimism-corrected performance values
        self.auc = None
        self.sen = None
        self.spe = None
        self.F1 = None
        self.mcc = None

        # self.df_fi = None  # feature importance dataframe

        self.dml = None
        self.effective = None
        self.component1 = None
        self.component2 = None
        self.corrected_fpr_list = []
        self.corrected_tpr_list = []

    def _calculate_optimism_corrected_pfm(self):
        # calculate mean and 95% confidence interval of optimism-corrected performance
        self.auc = get_performance_helper(app_pfm=self.original_apparent_performance.roc_auc,
                                          opti_distr=self.stat_opti_auc.get_formatted_mean_and_interval(
                                              deci=5, interval_type='p'
                                          ))
        self.sen = get_performance_helper(app_pfm=self.original_apparent_performance.sensitivity,
                                          opti_distr=self.stat_opti_sen.get_formatted_mean_and_interval(
                                              deci=5, interval_type='p'
                                          ))
        self.spe = get_performance_helper(app_pfm=self.original_apparent_performance.specificity,
                                          opti_distr=self.stat_opti_spe.get_formatted_mean_and_interval(
                                              deci=5, interval_type='p'
                                          ))
        self.F1 = get_performance_helper(app_pfm=self.original_apparent_performance.F1,
                                         opti_distr=self.stat_opti_F1.get_formatted_mean_and_interval(
                                             deci=5, interval_type='p'
                                         ))
        self.mcc = get_performance_helper(app_pfm=self.original_apparent_performance.mcc,
                                          opti_distr=self.stat_opti_mcc.get_formatted_mean_and_interval(
                                              deci=5, interval_type='p'
                                          ))
        if self.flq_prev is not None:
            self.dml = get_performance_helper(app_pfm=self.original_apparent_performance.receive_unnecessary_DML,
                                              opti_distr=self.stat_opti_dlm.get_formatted_mean_and_interval(
                                                  deci=5, interval_type='p'))
            self.effective = get_performance_helper(app_pfm=self.original_apparent_performance.receive_effective_regimen,
                                                    opti_distr=self.stat_opti_effective.get_formatted_mean_and_interval(
                                                        deci=5, interval_type='p'))
            self.component1 = get_performance_helper(app_pfm=self.original_apparent_performance.uComponent1,
                                                     opti_distr=self.stat_opti_uComponent1.get_formatted_mean_and_interval(
                                                         deci=5, interval_type='p'))
            self.component2 = get_performance_helper(app_pfm=self.original_apparent_performance.uComponent2,
                                                     opti_distr=self.stat_opti_uComponent2.get_formatted_mean_and_interval(
                                                         deci=5, interval_type='p'))
        if self.tradeoff is not None:
            original_app_utility = self.tradeoff * self.original_apparent_performance.uComponent1 \
                                   + self.original_apparent_performance.uComponent2
            self.utility = get_performance_helper(app_pfm=original_app_utility,
                                                  opti_distr=self.stat_opti_utility.get_formatted_mean_and_interval(
                                                     deci=5, interval_type='p'))
        if self.tradeoff_list is not None:
            uDic_origin = {}
            for i in self.tradeoff_list:
                i = int(i)
                uDic_origin['original apparent lambda {}'.format(i)] = \
                    i * self.original_apparent_performance.uComponent1 \
                    + self.original_apparent_performance.uComponent2
                self.utility_list_by_threshold.append(
                    get_performance_helper(app_pfm=uDic_origin['original apparent lambda {}'.format(i)],
                                           opti_distr=self.uDic[
                                               'trad-off threshold {}'.format(i)].get_formatted_mean_and_interval(
                                               deci=5, interval_type='p'))
                )
        if self.roc_curve:
            app_fpr_list = self.original_apparent_performance.fpr
            app_tpr_list = self.original_apparent_performance.tpr
            # print(len(app_fpr_list))
            # print(len(self.list_stat_opti_fpr))
            # print(self.list_stat_opti_fpr)
            for i in range(len(app_fpr_list)):
                fpr = get_performance_helper(
                    app_pfm=app_fpr_list[i],
                    opti_distr=self.list_stat_opti_fpr[i].get_formatted_mean_and_interval(deci=5, interval_type='p'))
                tpr = get_performance_helper(
                    app_pfm=app_tpr_list[i],
                    opti_distr=self.list_stat_opti_tpr[i].get_formatted_mean_and_interval(deci=5, interval_type='p'))
                self.corrected_fpr_list.append(fpr)
                self.corrected_tpr_list.append(tpr)
            # plot ROC curve
            self._plot_roc_curve()

    # def get_feature_importance(self, save_cvs_path=None):
    #     """ percent of times predictors were identified as significant """
    #     df = pd.DataFrame(self.predictor_counts)          # convert to dataframe
    #     self.df_fi = df.T                                 # transpose
    #     self.df_fi.columns = ['Counts']                   # rename column name
    #     self.df_fi = self.df_fi.sort_values('Counts', ascending=False)  # sort based on counts
    #     if save_cvs_path is not None:
    #         self.df_fi.to_csv(save_cvs_path)

    def plot_auc_optimism(self):
        # optimism for AUC values
        auc_optimism_list = [optimism.auc for optimism in self.optimism_list]
        # mean and 95% confidence interval for auc optimisms
        avg_and_ci_auc_optimism = self.stat_opti_auc.get_formatted_mean_and_interval(deci=2, interval_type='p')
        print('optimism and 95% CI:', avg_and_ci_auc_optimism)
        # values for x-axis
        x = np.linspace(start=1, stop=len(auc_optimism_list), num=len(auc_optimism_list))
        # plot the graph
        plt.plot(x, auc_optimism_list, color='#4b0082', lw=1, alpha=0.8)
        # average value of AUC optimism
        plt.axhline(y=self.stat_opti_auc.get_mean(), linestyle='-', alpha=0.8)
        plt.text((len(x) + (len(x) / 8)) / 2, min(auc_optimism_list),
                 "Avg. Optimism {}".format(avg_and_ci_auc_optimism),
                 ha="right", va="bottom")
        plt.xlabel('bootstrap numbers')
        plt.ylabel('optimism')
        plt.title('Bootstrap Optimisms')
        plt.show()

    def _plot_roc_curve(self):
        mean_fpr_list = []
        min_fpr_list = []
        max_fpr_list = []
        mean_tpr_list = []
        min_tpr_list = []
        max_tpr_list = []
        for i in range(len(self.corrected_tpr_list)):
            mean_fpr, min_fpr, max_fpr = get_mean_min_max(self.corrected_fpr_list)
            mean_tpr, min_tpr, max_tpr = get_mean_min_max(self.corrected_tpr_list)
            mean_fpr_list.append(mean_fpr)
            mean_tpr_list.append(mean_tpr)
            min_fpr_list.append(min_fpr)
            min_tpr_list.append(min_tpr)
            max_fpr_list.append(max_fpr)
            max_tpr_list.append(max_tpr)

        # print ROC curve
        plt.plot(mean_fpr_list, mean_tpr_list, color='lightblue', lw=0.5, alpha=0.4)
        plt.plot([0, 1], [0, 1], color='blue', lw=0.8, alpha=0.6, linestyle='--')
        plt.fill_between(x=mean_fpr_list, y1=min_tpr_list, y2=max_tpr_list, alpha=0.15, color='darkblue')
        plt.fill_between(x=mean_tpr_list, y1=min_fpr_list, y2=max_fpr_list, alpha=0.15, color='darkblue')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Logistic Regression + Permutation Importance')
        plt.show()


def performance_table_by_diff_features(df, y_name, classifier, feature_selection, predictor_names,
                                       num_bootstrap=200, filename=None,
                                       max_num_feature=None, min_num_features=None, max_penalty=None, min_penalty=None):
    """ explore relationship between number of features, feature selection method, and model performance """
    optimism_corrected_auc_list = []
    apparent_performance_auc_list = []
    num_features = []
    penalty_list = []
    if feature_selection == 'LASSO':
        num = round((max_penalty - min_penalty) / 0.01)
        for i in np.linspace(min_penalty, max_penalty, num + 1):
            print('penalty value:', i)
            bootstrap_model = BootstrapModel(df=df, y_name=y_name, classifier='logistic',
                                             feature_selection='LASSO', lasso_penalty=i)
            bootstrap_model.get_optimism_corrected_pfm(num_bootstrap=num_bootstrap, predictor_names=predictor_names)
            optimism_corrected_auc_list.append(bootstrap_model.corrected_pfm.auc)
            apparent_performance_auc_list.append(round(bootstrap_model.original_apparent_pfm.roc_auc, 3))
            num_features.append(len(bootstrap_model.select_significant_features(df=df)))
            penalty_list.append(i)
    elif feature_selection == 'PI' or 'RFE':
        for i in range(min_num_features, max_num_feature + 1):
            print('number of features selected:', i)
            bootstrap_model = BootstrapModel(df=df, y_name=y_name, classifier=classifier,
                                             feature_selection=feature_selection, num_selected_features=i)
            bootstrap_model.get_optimism_corrected_pfm(num_bootstrap=num_bootstrap, predictor_names=predictor_names)
            optimism_corrected_auc_list.append(bootstrap_model.corrected_pfm.auc)
            apparent_performance_auc_list.append(round(bootstrap_model.original_apparent_pfm.roc_auc, 3))
            num_features.append(i)
            penalty_list.append('not applicable')
    else:
        raise ValueError('feature selection method not included in this study')
    performance_dict = {'optimism-corrected performance': optimism_corrected_auc_list,
                        'apparent performance': apparent_performance_auc_list,
                        'number of features': num_features,
                        'penalty strength': penalty_list}
    df_performance = pd.DataFrame(performance_dict)
    print(df_performance)
    if filename is not None:
        df_performance.to_csv("../data/feature_performance_table/{}.csv".format(filename))

ModuleNotFoundError: No module named 'source'